<a href="https://colab.research.google.com/github/imjunyoung404/ai-class-3-2/blob/main/%EA%B8%B0%ED%95%99%ED%94%84%EA%B3%BC%EC%A0%9C1_%EC%BB%B4%ED%93%A8%ED%84%B0%EA%B3%B5%ED%95%99%EA%B3%BC_202035220_%EC%9E%84%EC%A4%80%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [14]:
# 1. 데이터셋 불러오기
# Iris 데이터셋을 불러오고 입력 데이터(X)와 타깃(y)으로 나눕니다.
iris_data = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/R/iris.csv')
X = iris_data[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']].to_numpy()
y = iris_data['Species'].to_numpy()

In [15]:
# 2. 데이터셋 분리
# 데이터를 훈련 세트(70%)와 테스트 세트(30%)로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)


In [17]:
# 3. 데이터 표준화
# StandardScaler를 사용해 훈련 세트와 테스트 세트의 입력 데이터를 표준화합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# 4. 하이퍼파라미터 딕셔너리 생성
# 학습을 위한 하이퍼파라미터 범위를 지정합니다. 여기서는 랜덤 서치를 사용하기 위해 scipy의 uniform과 randint를 사용하여 값의 범위를 설정합니다.
from scipy.stats import uniform
from scipy.stats import randint

hyperparameters = {
    #'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],  # 학습률 스케줄 (비활성화됨)
    #'eta0': uniform(0.01, 1.0),  # 초기 학습률 (비활성화됨)
    'max_iter': randint(50, 101),  # 반복 횟수를 randint로 설정
    'penalty': ['l1','l2', 'elasticnet'],  # 정규화 방식 (L1, L2, ElasticNet 중에서 선택)
    'alpha': uniform(0.0001, 0.1),  # 정규화 강도 (alpha 값을 uniform 분포로 설정)
    'loss': ['hinge', 'log_loss', 'modified_huber'],  # 손실 함수 (hinge, log_loss, modified_huber 중에서 선택)
}


In [19]:
# 5. RandomizedSearchCV 설정
# SGDClassifier에 대해 랜덤 서치를 사용하여 최적의 하이퍼파라미터를 탐색합니다. n_jobs=-1은 모든 CPU 코어를 사용하고, n_iter=200은 200번의 랜덤 조합을 시도합니다.
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
rs = RandomizedSearchCV(SGDClassifier(random_state=2024), hyperparameters, n_jobs=-1, n_iter=200)


In [20]:
# 6. 모델 학습
# 훈련 데이터를 사용해 랜덤 서치로 최적의 하이퍼파라미터를 찾고, 모델을 학습합니다.
rs.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(estimator=SGDClassifier(random_state=2024), n_iter=200,
                   n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7df9a1326290>,
                                        'loss': ['hinge', 'log_loss',
                                                 'modified_huber'],
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7df9a1326140>,
                                        'penalty': ['l1', 'l2', 'elasticnet']})

In [21]:
# 7. 최고 교차 검증 점수 출력
# 교차 검증에서의 최고 평균 테스트 점수를 출력합니다.
import numpy as np
print(np.max(rs.cv_results_['mean_test_score']))

0.980952380952381


In [22]:
# 8. 최적의 하이퍼파라미터와 모델 성능 출력
# 랜덤 서치를 통해 찾은 최적의 하이퍼파라미터를 출력합니다.
print("최적 하이퍼파라미터: ", rs.best_params_)

최적 하이퍼파라미터:  {'alpha': 0.025932850154109177, 'loss': 'modified_huber', 'max_iter': 50, 'penalty': 'l1'}


In [23]:
# 9. 최적의 SGDClassifier로 모델 학습
# 랜덤 서치에서 찾은 최적의 SGDClassifier로 다시 훈련 세트에서 학습을 진행합니다.
sc = rs.best_estimator_

In [24]:
# 10. 최적 모델로 학습
# 훈련 데이터를 사용해 최적의 SGDClassifier 모델을 학습합니다.
sc.fit(X_train_scaled, y_train)

SGDClassifier(alpha=0.025932850154109177, loss='modified_huber', max_iter=50,
              penalty='l1', random_state=2024)

In [25]:
# 11. 훈련 및 테스트 세트 성능 출력
# 훈련 세트와 테스트 세트에서 모델의 성능(정확도)을 출력합니다.
train_score = sc.score(X_train_scaled, y_train)
test_score = sc.score(X_test_scaled, y_test)

# 12. 훈련과 테스트 성능 차이 계산 및 출력
# 훈련 세트와 테스트 세트의 성능 차이를 계산하고 출력합니다.
print(train_score)
print(test_score)
print(train_score - test_score)

0.9809523809523809
0.9777777777777777
0.0031746031746031633
